In [1]:
# =========================
# HELPER
# =========================

EMPTY = "."
PLAYER_X = "X"
PLAYER_O = "O"


def print_board(board):
    """In bàn cờ"""
    size = len(board)
    print("\n  " + " ".join(str(i) for i in range(size)))
    for i in range(size):
        print(i, " ".join(board[i]))
    print()


def get_available_moves(board):
    """Lấy các ô trống"""
    return [(i, j) for i in range(len(board))
                    for j in range(len(board))
                    if board[i][j] == EMPTY]


def check_winner(board, player, win_len=3):
    """Kiểm tra thắng"""
    n = len(board)

    # Hàng & cột
    for i in range(n):
        for j in range(n - win_len + 1):
            if all(board[i][j+k] == player for k in range(win_len)):
                return True
            if all(board[j+k][i] == player for k in range(win_len)):
                return True

    # Chéo chính
    for i in range(n - win_len + 1):
        for j in range(n - win_len + 1):
            if all(board[i+k][j+k] == player for k in range(win_len)):
                return True

    # Chéo phụ
    for i in range(n - win_len + 1):
        for j in range(win_len - 1, n):
            if all(board[i+k][j-k] == player for k in range(win_len)):
                return True

    return False


def is_draw(board):
    """Kiểm tra hòa"""
    return all(cell != EMPTY for row in board for cell in row)


In [2]:
# =========================
# CORE – MINIMAX + ALPHA-BETA
# =========================

def evaluate(board, ai, human):
    if check_winner(board, ai):
        return 10
    if check_winner(board, human):
        return -10
    return 0


def minimax(board, depth, alpha, beta, is_maximizing, ai, human):
    score = evaluate(board, ai, human)

    if score != 0 or is_draw(board) or depth == 0:
        return score

    if is_maximizing:
        best = -float("inf")
        for (i, j) in get_available_moves(board):
            board[i][j] = ai
            val = minimax(board, depth - 1, alpha, beta, False, ai, human)
            board[i][j] = EMPTY
            best = max(best, val)
            alpha = max(alpha, val)
            if beta <= alpha:
                break
        return best
    else:
        best = float("inf")
        for (i, j) in get_available_moves(board):
            board[i][j] = human
            val = minimax(board, depth - 1, alpha, beta, True, ai, human)
            board[i][j] = EMPTY
            best = min(best, val)
            beta = min(beta, val)
            if beta <= alpha:
                break
        return best


def best_move(board, depth, ai, human):
    best_score = -float("inf")
    move = None

    for (i, j) in get_available_moves(board):
        board[i][j] = ai
        score = minimax(board, depth, -float("inf"), float("inf"),
                        False, ai, human)
        board[i][j] = EMPTY

        if score > best_score:
            best_score = score
            move = (i, j)

    return move



In [ ]:
# =========================
# MAIN
# =========================

def play_game():
    print("CỜ CARO - MINIMAX + ALPHA-BETA")

    while True:
        # nhap kich thuoc ban co
        while True:
            n = input("\nNhập kích thước bàn cờ (n >= 3): ")
            if n.isdigit() and int(n) >= 3:
                n = int(n)
                break
            print("Nhập sai! Vui lòng nhập số >= 3.")

        # tao ban co moi
        while True:
            board = [[EMPTY for _ in range(n)] for _ in range(n)]

            # -------- CHỌN LƯỢT --------
            while True:
                print("\n---- LƯỢT ĐI ----")
                print("1. Bạn đi trước")
                print("2. AI đi trước")
                print("-------------------")
                turn_choice = input("Chọn (1/2): ")
                if turn_choice in ["1", "2"]:
                    break
                print("Chọn sai!")

            # -------- CHỌN QUÂN --------
            while True:
                print("\n----- QUÂN -------")
                print("1. X")
                print("2. O")
                print("-------------------")
                piece_choice = input("Chọn (1/2): ")
                if piece_choice in ["1", "2"]:
                    break
                print("Chọn sai!")

            # Gán quân
            if piece_choice == "1":
                human, ai = PLAYER_X, PLAYER_O
            else:
                human, ai = PLAYER_O, PLAYER_X

            # Gán lượt đầu
            current_player = human if turn_choice == "1" else ai

            print("\n--- THÔNG TIN ---")
            print(f"Bàn cờ: {n} x {n}")
            print(f"Bạn : {human}")
            print(f"AI  : {ai}")
            print(f"Đi trước: {'Bạn' if current_player == human else 'AI'}")
            
            # thuc hien ván game
            while True:
                print_board(board)

                if current_player == human:
                    # Người chơi
                    while True:
                        try:
                            x, y = map(int, input("Nhập nước đi (row col): ").split())
                            if 0 <= x < n and 0 <= y < n and board[x][y] == EMPTY:
                                break
                            print("Nước đi không hợp lệ!")
                        except:
                            print("Nhập sai!")

                    board[x][y] = human

                    if check_winner(board, human):
                        print_board(board)
                        print("Bạn thắng!")
                        break

                    current_player = ai

                else:
                    # AI
                    print("AI đang suy nghĩ...")
                    move = best_move(board, depth=5, ai=ai, human=human)#depth = 5 độ sâu tìm kiếm
                    if move:
                        board[move[0]][move[1]] = ai
                        print(f"AI đánh: {move}")

                    if check_winner(board, ai):
                        print_board(board)
                        print("AI thắng!")
                        break

                    current_player = human

                if is_draw(board):
                    print_board(board)
                    print("Hòa!")
                    break

            # =========================
            # MENU SAU KHI KẾT THÚC VÁN
            # =========================
            print("\n--- BẠN MUỐN LÀM GÌ TIẾP? ---")
            print("1. Chơi lại (giữ nguyên bàn cờ)")
            print("2. Chọn bàn cờ mới")
            print("0. Thoát")

            choice = input("Chọn (0/1/2): ")

            if choice == "1":
                # chơi lại với cùng n
                continue
            elif choice == "2":
                # thoát vòng n, nhập n mới
                break
            elif choice == "0":
                print("Kết thúc chương trình.")
                return
            else:
                print("Lựa chọn không hợp lệ! Quay lại menu.")



# CHẠY GAME
play_game()

CỜ CARO - MINIMAX + ALPHA-BETA

---- LƯỢT ĐI ----
1. Bạn đi trước
2. AI đi trước
-------------------

----- QUÂN -------
1. X
2. O
-------------------

--- THÔNG TIN ---
Bàn cờ: 3 x 3
Bạn : X
AI  : O
Đi trước: Bạn

  0 1 2
0 . . .
1 . . .
2 . . .


  0 1 2
0 . . .
1 . X .
2 . . .

AI đang suy nghĩ...
AI đánh: (0, 0)

  0 1 2
0 O . .
1 . X .
2 . . .


  0 1 2
0 O X .
1 . X .
2 . . .

AI đang suy nghĩ...
AI đánh: (2, 1)

  0 1 2
0 O X .
1 . X .
2 . O .


  0 1 2
0 O X .
1 X X .
2 . O .

AI đang suy nghĩ...
AI đánh: (1, 2)

  0 1 2
0 O X .
1 X X O
2 . O .


  0 1 2
0 O X X
1 X X O
2 . O .

AI đang suy nghĩ...
AI đánh: (2, 0)

  0 1 2
0 O X X
1 X X O
2 O O .


  0 1 2
0 O X X
1 X X O
2 O O X

Hòa!

--- BẠN MUỐN LÀM GÌ TIẾP? ---
1. Chơi lại (giữ nguyên bàn cờ)
2. Chọn bàn cờ mới
0. Thoát

---- LƯỢT ĐI ----
1. Bạn đi trước
2. AI đi trước
-------------------

----- QUÂN -------
1. X
2. O
-------------------

--- THÔNG TIN ---
Bàn cờ: 5 x 5
Bạn : O
AI  : X
Đi trước: Bạn

  0 1 2 3 4
0 . . . . 